# John Hopkins Corona Daten

COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University
https://github.com/CSSEGISandData/COVID-19

Link zu den Daten bekommen:
- Datei raussuchen: 
    - **COVID-19** / **csse_covid_19_data** / **csse_covid_19_time_series** / 
- Datei `time_series_covid19_confirmed_global.csv` anklicken und Link von `RAW` (rechts oben) kopieren und diesen verwenden


Einlesen analog Vorgehen hier: https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6

## `confirmed` einlesen

In [1]:
import pandas as pd


data_confirmed="https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"

df_confirmed_wide = pd.read_csv(data_confirmed)
df_confirmed_wide

ModuleNotFoundError: No module named 'pandas'

Die Daten liegen mit jedem Datum pro Spalte vor. Diese Art ist für die Verarbeitung mit `Pandas` sehr ungeeignet, weshalb wir aus dem **breiten** (nach rechts) Dataframe jetzt einen **langen** (nach unten) erstellen und nutzen dafür den `.melt` Befehl

In [2]:
# Liste der Einträge aller Daten erstellen 
dates = df_confirmed_wide.columns[4:] # Nimm alle Spalten ab der fünften (0=erste Spalte)
dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '1/22/21', '1/23/21', '1/24/21', '1/25/21', '1/26/21', '1/27/21',
       '1/28/21', '1/29/21', '1/30/21', '1/31/21'],
      dtype='object', length=376)

In [3]:
df_confirmed_long = df_confirmed_wide.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
df_confirmed_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
102643,NaN,Vietnam,14.058324,108.277199,1/31/21,1817
102644,NaN,West Bank and Gaza,31.952200,35.233200,1/31/21,158962
102645,NaN,Yemen,15.552727,48.516388,1/31/21,2121
102646,NaN,Zambia,-13.133897,27.849332,1/31/21,54217


- der neue lange Dataframe `df_long` hat **100191 rows × 6 columns** = 601.146 Einträge
- der alte breite Dataframe `df_wide` hat **273 rows × 371 columns** = 101.283 Einträge

Die Einträge haben sich also für unseren neuen Dataframe `df_long` vervielfacht, da jetzt das Datum immer dazu geschrieben werden muss und somit oft doppelt vorkommt

### `deaths` + `recovered` analog 

In [4]:
data_death="https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
data_recovered="https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"


df_death_wide = pd.read_csv(data_death)
df_recovered_wide = pd.read_csv(data_recovered)

df_death_long = df_death_wide.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)

recovered_df_long = df_recovered_wide.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)


#### alle drei Dataframes zusammenführen

In [5]:
# Merging confirmed_df_long and deaths_df_long
full_table = df_confirmed_long.merge(
  right=df_death_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)
# Merging full_table and recovered_df_long
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0.0
...,...,...,...,...,...,...,...,...
102643,NaN,Vietnam,14.058324,108.277199,1/31/21,1817,35,1457.0
102644,NaN,West Bank and Gaza,31.952200,35.233200,1/31/21,158962,1833,148993.0
102645,NaN,Yemen,15.552727,48.516388,1/31/21,2121,615,1426.0
102646,NaN,Zambia,-13.133897,27.849332,1/31/21,54217,763,48000.0


#### Datum als Datetime

In [6]:
full_table['Date'] = pd.to_datetime(full_table['Date'])
full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0.0
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0


#### missing NaN 

In [7]:
full_table.isna().sum()

Province/State    71064
Country/Region        0
Lat                 376
Long                376
Date                  0
Confirmed             0
Deaths                0
Recovered          7896
dtype: int64

#### recovered mit nullen füllen

In [8]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)

#### Kreuzfahrtschiffe rausnehmen

In [9]:
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | full_table['Province/State'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('MS Zaandam')
full_ship = full_table[ship_rows]

full_table = full_table[~(ship_rows)]

### Neue Spalte `active`

In [10]:
# Active Case = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

### Länder zusammenfassen (falls mehrere Province/State) infos vorhanden

In [11]:
full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
71435,2021-01-31,Vietnam,1817,35,1457.0,325.0
71436,2021-01-31,West Bank and Gaza,158962,1833,148993.0,8136.0
71437,2021-01-31,Yemen,2121,615,1426.0,80.0
71438,2021-01-31,Zambia,54217,763,48000.0,5454.0


### neue Spalten New cases, New deaths and New recovered

In [12]:
import numpy as np 

# new cases 
temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan
# renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']
# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])
# filling na with 0
full_grouped = full_grouped.fillna(0)
# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')
# 
full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

In [13]:
full_grouped.to_csv("JH_Corona_global.csv", index=False)

## Deutschland rausfiltern

In [17]:
Deutschland = full_grouped.loc[full_grouped["Country/Region"] == "Germany"]
Deutschland.to_csv("JH_Corona_Deutschland.csv", index=False)